In [1]:
#!/usr/bin/env python3
"""
Complete ZSim Metrics Extractor
Extracts ALL available metrics from ZSim output files for both Host and PIM configurations
"""
import os
import csv
import glob
from pathlib import Path

def parse_zsim_file(zsim_file):
    """Parse a single zsim file and return ALL metrics"""
    
    # Initialize all metrics
    instructions = cycles = 0
    cycles_list = []
    
    # Core metrics
    uops = branch_uops = fp_add_sub_uops = fp_mul_div_uops = 0
    bbls = approx_instrs = mispred_branches = mispred_instrs = 0
    mispred_penalty = op_executed = 0
    load_stalls = store_stalls = 0
    spatial_locality = temporal_locality = 0
    c_cycles = 0  # Contention cycles
    
    # Cache metrics - L1I
    l1i_hits = l1i_misses = l1i_fhits = 0
    l1i_shared_requests = 0
    
    # Cache metrics - L1D
    l1d_gets_hits = l1d_getx_hits = 0
    l1d_gets_misses = l1d_getx_misses = 0
    l1d_fhgets = l1d_fhgetx = 0
    l1d_puts = l1d_putx = 0
    l1d_inv = l1d_invx = l1d_fwd = 0
    l1d_lat_getnl = l1d_lat_getnet = 0
    l1d_shared_requests = 0
    
    # Cache metrics - L2
    l2_gets_hits = l2_getx_hits = 0
    l2_gets_misses = l2_getx_im_misses = l2_getx_sm_misses = 0
    l2_puts = l2_putx = 0
    l2_inv = l2_invx = l2_fwd = 0
    l2_lat_getnl = l2_lat_getnet = 0
    l2_shared_requests = 0
    
    # Cache metrics - L3
    l3_gets_hits = l3_getx_hits = 0
    l3_gets_misses = l3_getx_im_misses = l3_getx_sm_misses = 0
    l3_puts = l3_putx = 0
    l3_inv = l3_invx = l3_fwd = 0
    l3_lat_getnl = l3_lat_getnet = 0
    l3_shared_requests = 0
    
    # Memory controller stats
    mem_rd = mem_wr = 0
    mem_rdlat = mem_wrlat = 0
    mem_reissued = 0
    
    # Weave simulation time
    weave_time = 0
    
    # State flags
    l1i = l1d = l2d = l3d = False
    in_mem = False
    in_contention = False
    
    # Energy constants
    ENERGY_L1I_HIT, ENERGY_L1I_MISS = 15.0, 33.0
    ENERGY_L1D_HIT, ENERGY_L1D_MISS = 15.0, 33.0
    ENERGY_L2_HIT, ENERGY_L2_MISS = 46.0, 93.0
    ENERGY_L3_HIT, ENERGY_L3_MISS = 945.0, 1904.0
    ENERGY_MAIN_MEMORY = 2.0
    ENERGY_PER_CYCLE = 0.1
    CLOCK_FREQUENCY_GHz = 5.2  # Example: 5.2 GHz
    
    # Parse the file
    with open(zsim_file, "r") as ins:
        for line in ins:
            try:
                # Weave simulation time
                if "time:" in line and "# Weave simulation time" in line:
                    weave_time = int(line.split()[1])
                
                # Core Metrics
                if "instrs: " in line and "# Simulated instructions" in line:
                    instructions += int(line.split()[1])
                if "Simulated unhalted cycles" in line:
                    cycles_list.append(int(line.split()[1]))
                if "cCycles:" in line and "# Cycles due to contention" in line:
                    c_cycles += int(line.split()[1])
                if "uops:" in line and "# Retired micro-ops" in line:
                    uops += int(line.split()[1])
                if "branchUops:" in line:
                    branch_uops += int(line.split()[1])
                if "fpAddSubUops:" in line:
                    fp_add_sub_uops += int(line.split()[1])
                if "fpMulDivUops:" in line:
                    fp_mul_div_uops += int(line.split()[1])
                if "bbls:" in line and "# Basic blocks" in line:
                    bbls += int(line.split()[1])
                if "approxInstrs:" in line:
                    approx_instrs += int(line.split()[1])
                if "mispredBranches:" in line:
                    mispred_branches += int(line.split()[1])
                if "mispredInstrs:" in line:
                    mispred_instrs += int(line.split()[1])
                if "mispredPenalty:" in line:
                    mispred_penalty += int(line.split()[1])
                if "opExecuted:" in line:
                    op_executed += int(line.split()[1])
                if "loadStallsTotal:" in line:
                    load_stalls += int(line.split()[1])
                if "storeStallsTotal:" in line:
                    store_stalls += int(line.split()[1])
                if "spatialLocality:" in line:
                    spatial_locality += int(line.split()[1])
                if "temporalLocality:" in line:
                    temporal_locality += int(line.split()[1])
                
                # Cache level detection
                if "l1i:" in line or "l1i-0:" in line:
                    l1i, l1d, l2d, l3d = True, False, False, False
                elif "l1d:" in line or "l1d-0:" in line:
                    l1d, l1i, l2d, l3d = True, False, False, False
                elif "l2:" in line or "l2-0:" in line:
                    l2d, l1d, l1i, l3d = True, False, False, False
                elif ("l3:" in line or "l3-0b0:" in line) and "Cache stats" in line:
                    l3d, l1d, l1i, l2d = True, False, False, False
                elif "mem:" in line and "Memory controller" in line:
                    in_mem = True
                    l1d = l1i = l2d = l3d = False
                elif "sched: # Scheduler stats" in line:
                    l1d = l1i = l2d = l3d = in_mem = False
                
                # L1I Cache stats
                if l1i:
                    if "fhGETS:" in line:
                        l1i_fhits += int(line.split()[1])
                    elif "hGETS:" in line and "# GETS hits" in line:
                        l1i_hits += int(line.split()[1])
                    elif "mGETS:" in line and "# GETS misses" in line:
                        l1i_misses += int(line.split()[1])
                    elif "sharedRequests:" in line:
                        l1i_shared_requests += int(line.split()[1])
                
                # L1D Cache stats
                if l1d:
                    if "fhGETS:" in line:
                        l1d_fhgets += int(line.split()[1])
                    elif "fhGETX:" in line:
                        l1d_fhgetx += int(line.split()[1])
                    elif "hGETS:" in line and "# GETS hits" in line:
                        l1d_gets_hits += int(line.split()[1])
                    elif "hGETX:" in line and "# GETX hits" in line:
                        l1d_getx_hits += int(line.split()[1])
                    elif "mGETS:" in line and "# GETS misses" in line:
                        l1d_gets_misses += int(line.split()[1])
                    elif "mGETXIM:" in line:
                        l1d_getx_misses += int(line.split()[1])
                    elif "PUTS:" in line and "# Clean evictions" in line:
                        l1d_puts += int(line.split()[1])
                    elif "PUTX:" in line and "# Dirty evictions" in line:
                        l1d_putx += int(line.split()[1])
                    elif "INV:" in line and "# Invalidates" in line:
                        l1d_inv += int(line.split()[1])
                    elif "INVX:" in line and "# Downgrades" in line:
                        l1d_invx += int(line.split()[1])
                    elif "FWD:" in line and "# Forwards" in line:
                        l1d_fwd += int(line.split()[1])
                    elif "latGETnl:" in line:
                        l1d_lat_getnl += int(line.split()[1])
                    elif "latGETnet:" in line:
                        l1d_lat_getnet += int(line.split()[1])
                    elif "sharedRequests:" in line:
                        l1d_shared_requests += int(line.split()[1])
                
                # L2 Cache stats
                if l2d:
                    if "hGETS:" in line and "# GETS hits" in line:
                        l2_gets_hits += int(line.split()[1])
                    elif "hGETX:" in line and "# GETX hits" in line:
                        l2_getx_hits += int(line.split()[1])
                    elif "mGETS:" in line and "# GETS misses" in line:
                        l2_gets_misses += int(line.split()[1])
                    elif "mGETXIM:" in line:
                        l2_getx_im_misses += int(line.split()[1])
                    elif "mGETXSM:" in line:
                        l2_getx_sm_misses += int(line.split()[1])
                    elif "PUTS:" in line and "# Clean evictions" in line:
                        l2_puts += int(line.split()[1])
                    elif "PUTX:" in line and "# Dirty evictions" in line:
                        l2_putx += int(line.split()[1])
                    elif "INV:" in line and "# Invalidates" in line:
                        l2_inv += int(line.split()[1])
                    elif "INVX:" in line and "# Downgrades" in line:
                        l2_invx += int(line.split()[1])
                    elif "FWD:" in line and "# Forwards" in line:
                        l2_fwd += int(line.split()[1])
                    elif "latGETnl:" in line:
                        l2_lat_getnl += int(line.split()[1])
                    elif "latGETnet:" in line:
                        l2_lat_getnet += int(line.split()[1])
                    elif "sharedRequests:" in line:
                        l2_shared_requests += int(line.split()[1])
                
                # L3 Cache stats
                if l3d:
                    if "hGETS:" in line and "# GETS hits" in line:
                        l3_gets_hits += int(line.split()[1])
                    elif "hGETX:" in line and "# GETX hits" in line:
                        l3_getx_hits += int(line.split()[1])
                    elif "mGETS:" in line and "# GETS misses" in line:
                        l3_gets_misses += int(line.split()[1])
                    elif "mGETXIM:" in line:
                        l3_getx_im_misses += int(line.split()[1])
                    elif "mGETXSM:" in line:
                        l3_getx_sm_misses += int(line.split()[1])
                    elif "PUTS:" in line and "# Clean evictions" in line:
                        l3_puts += int(line.split()[1])
                    elif "PUTX:" in line and "# Dirty evictions" in line:
                        l3_putx += int(line.split()[1])
                    elif "INV:" in line and "# Invalidates" in line:
                        l3_inv += int(line.split()[1])
                    elif "INVX:" in line and "# Downgrades" in line:
                        l3_invx += int(line.split()[1])
                    elif "FWD:" in line and "# Forwards" in line:
                        l3_fwd += int(line.split()[1])
                    elif "latGETnl:" in line:
                        l3_lat_getnl += int(line.split()[1])
                    elif "latGETnet:" in line:
                        l3_lat_getnet += int(line.split()[1])
                    elif "sharedRequests:" in line:
                        l3_shared_requests += int(line.split()[1])
                
                # Memory controller stats
                if in_mem:
                    if "rd:" in line and "# Read requests" in line:
                        mem_rd += int(line.split()[1])
                    elif "wr:" in line and "# Write requests" in line:
                        mem_wr += int(line.split()[1])
                    elif "rdlat:" in line:
                        mem_rdlat += int(line.split()[1])
                    elif "wrlat:" in line:
                        mem_wrlat += int(line.split()[1])
                    elif "reissuedAccesses:" in line:
                        mem_reissued += int(line.split()[1])
                        
            except:
                continue
    
    # Calculate derived metrics
    cycles = max(cycles_list) if cycles_list else 0
    ipc = float(instructions) / float(cycles) if cycles > 0 else 0.0
    
    # Total cache hits/misses
    l1_total_hits = l1d_gets_hits + l1d_getx_hits
    l1_total_misses = l1d_gets_misses + l1d_getx_misses
    l2_total_hits = l2_gets_hits + l2_getx_hits
    l2_total_misses = l2_gets_misses + l2_getx_im_misses + l2_getx_sm_misses
    l3_total_hits = l3_gets_hits + l3_getx_hits
    l3_total_misses = l3_gets_misses + l3_getx_im_misses + l3_getx_sm_misses
    
    # Miss rates
    l1i_miss_rate = (l1i_misses / float(l1i_misses + l1i_hits)) * 100.0 if (l1i_misses + l1i_hits) > 0 else 0.0
    l1d_miss_rate = (l1_total_misses / float(l1_total_misses + l1_total_hits)) * 100.0 if (l1_total_misses + l1_total_hits) > 0 else 0.0
    l1_miss_rate = ((l1i_misses + l1_total_misses) / float(l1i_misses + l1i_hits + l1_total_misses + l1_total_hits)) * 100.0 if (l1i_misses + l1i_hits + l1_total_misses + l1_total_hits) > 0 else 0.0
    l2_miss_rate = (l2_total_misses / float(l2_total_misses + l2_total_hits)) * 100.0 if (l2_total_misses + l2_total_hits) > 0 else 0.0
    l3_miss_rate = (l3_total_misses / float(l3_total_misses + l3_total_hits)) * 100.0 if (l3_total_misses + l3_total_hits) > 0 else 0.0
    
    # MPKI (Misses Per Kilo Instructions)
    l1_mpki = (l1i_misses + l1_total_misses) / float(instructions / 1000.0) if instructions > 0 else 0.0
    l2_mpki = l2_total_misses / float(instructions / 1000.0) if instructions > 0 else 0.0
    l3_mpki = l3_total_misses / float(instructions / 1000.0) if instructions > 0 else 0.0
    
    # LFMR (Last Level Miss Ratio)
    lfmr = float(l3_total_misses) / float(l1i_misses + l1_total_misses) if (l1i_misses + l1_total_misses) > 0 else 0.0
    
    # Energy calculations
    energy_l1i = (l1i_hits * ENERGY_L1I_HIT) + (l1i_misses * ENERGY_L1I_MISS)
    energy_l1d = (l1_total_hits * ENERGY_L1D_HIT) + (l1_total_misses * ENERGY_L1D_MISS)
    energy_l2 = (l2_total_hits * ENERGY_L2_HIT) + (l2_total_misses * ENERGY_L2_MISS)
    energy_l3 = (l3_total_hits * ENERGY_L3_HIT) + (l3_total_misses * ENERGY_L3_MISS)
    energy_memory = (mem_rd+mem_wr)* ENERGY_MAIN_MEMORY
    energy_cpu_core = cycles * ENERGY_PER_CYCLE
    
    total_cache_energy = energy_l1i + energy_l1d + energy_l2 + energy_l3
    total_system_energy = total_cache_energy + energy_memory + energy_cpu_core
    
    # Power calculations
    execution_time_seconds = cycles / (CLOCK_FREQUENCY_GHz * 1e9)
    average_power_watts = (total_system_energy * 1e-12) / execution_time_seconds if execution_time_seconds > 0 else 0.0
    perf_power_ratio = ipc / average_power_watts if average_power_watts > 0 else 0.0
    
    # Branch prediction accuracy
    branch_pred_accuracy = ((branch_uops - mispred_branches) / float(branch_uops)) * 100.0 if branch_uops > 0 else 0.0
    
    # Average memory latency
    avg_mem_read_lat = mem_rdlat / float(mem_rd) if mem_rd > 0 else 0.0
    avg_mem_write_lat = mem_wrlat / float(mem_wr) if mem_wr > 0 else 0.0
    
    return {
        # Basic execution metrics
        'instructions': instructions,
        'cycles': cycles,
        'ipc': round(ipc, 4),
        'execution_time_s': round(execution_time_seconds, 6),
        'execution_time_us': round(execution_time_seconds * 1e6, 2),
        'weave_time': weave_time,
        
        # Core performance metrics
        'uops': uops,
        'branch_uops': branch_uops,
        'fp_add_sub_uops': fp_add_sub_uops,
        'fp_mul_div_uops': fp_mul_div_uops,
        'bbls': bbls,
        'approx_instrs': approx_instrs,
        'contention_cycles': c_cycles,
        'contention_percentage': round((c_cycles / float(cycles)) * 100, 2) if cycles > 0 else 0.0,
        
        # Branch prediction metrics
        'mispred_branches': mispred_branches,
        'mispred_instrs': mispred_instrs,
        'mispred_penalty': mispred_penalty,
        'branch_pred_accuracy': round(branch_pred_accuracy, 2),
        'avg_mispred_penalty': round(mispred_penalty / float(mispred_branches), 2) if mispred_branches > 0 else 0.0,
        
        # Stall metrics
        'load_stalls': load_stalls,
        'store_stalls': store_stalls,
        'total_stalls': load_stalls + store_stalls,
        'load_stall_percentage': round((load_stalls / float(cycles)) * 100, 2) if cycles > 0 else 0.0,
        'store_stall_percentage': round((store_stalls / float(cycles)) * 100, 2) if cycles > 0 else 0.0,
        
        # Locality metrics
        'spatial_locality': spatial_locality,
        'temporal_locality': temporal_locality,
        
        # L1I Cache metrics
        'l1i_hits': l1i_hits,
        'l1i_misses': l1i_misses,
        'l1i_filtered_hits': l1i_fhits,
        'l1i_total_accesses': l1i_hits + l1i_misses,
        'l1i_miss_rate': round(l1i_miss_rate, 2),
        'l1i_hit_rate': round(100 - l1i_miss_rate, 2),
        'l1i_shared_requests': l1i_shared_requests,
        
        # L1D Cache metrics
        'l1d_gets_hits': l1d_gets_hits,
        'l1d_getx_hits': l1d_getx_hits,
        'l1d_total_hits': l1_total_hits,
        'l1d_gets_misses': l1d_gets_misses,
        'l1d_getx_misses': l1d_getx_misses,
        'l1d_total_misses': l1_total_misses,
        'l1d_filtered_gets': l1d_fhgets,
        'l1d_filtered_getx': l1d_fhgetx,
        'l1d_total_accesses': l1_total_hits + l1_total_misses,
        'l1d_miss_rate': round(l1d_miss_rate, 2),
        'l1d_hit_rate': round(100 - l1d_miss_rate, 2),
        'l1d_puts': l1d_puts,
        'l1d_putx': l1d_putx,
        'l1d_inv': l1d_inv,
        'l1d_invx': l1d_invx,
        'l1d_fwd': l1d_fwd,
        'l1d_lat_getnl': l1d_lat_getnl,
        'l1d_lat_getnet': l1d_lat_getnet,
        'l1d_shared_requests': l1d_shared_requests,
        
        # Combined L1 metrics
        'l1_total_hits': l1i_hits + l1_total_hits,
        'l1_total_misses': l1i_misses + l1_total_misses,
        'l1_total_accesses': l1i_hits + l1i_misses + l1_total_hits + l1_total_misses,
        'l1_miss_rate': round(l1_miss_rate, 2),
        'l1_hit_rate': round(100 - l1_miss_rate, 2),
        'l1_mpki': round(l1_mpki, 2),
        
        # L2 Cache metrics
        'l2_gets_hits': l2_gets_hits,
        'l2_getx_hits': l2_getx_hits,
        'l2_total_hits': l2_total_hits,
        'l2_gets_misses': l2_gets_misses,
        'l2_getx_im_misses': l2_getx_im_misses,
        'l2_getx_sm_misses': l2_getx_sm_misses,
        'l2_total_misses': l2_total_misses,
        'l2_total_accesses': l2_total_hits + l2_total_misses,
        'l2_miss_rate': round(l2_miss_rate, 2),
        'l2_hit_rate': round(100 - l2_miss_rate, 2),
        'l2_mpki': round(l2_mpki, 2),
        'l2_puts': l2_puts,
        'l2_putx': l2_putx,
        'l2_inv': l2_inv,
        'l2_invx': l2_invx,
        'l2_fwd': l2_fwd,
        'l2_lat_getnl': l2_lat_getnl,
        'l2_lat_getnet': l2_lat_getnet,
        'l2_shared_requests': l2_shared_requests,
        
        # L3 Cache metrics
        'l3_gets_hits': l3_gets_hits,
        'l3_getx_hits': l3_getx_hits,
        'l3_total_hits': l3_total_hits,
        'l3_gets_misses': l3_gets_misses,
        'l3_getx_im_misses': l3_getx_im_misses,
        'l3_getx_sm_misses': l3_getx_sm_misses,
        'l3_total_misses': l3_total_misses,
        'l3_total_accesses': l3_total_hits + l3_total_misses,
        'l3_miss_rate': round(l3_miss_rate, 2),
        'l3_hit_rate': round(100 - l3_miss_rate, 2),
        'l3_mpki': round(l3_mpki, 2),
        'l3_puts': l3_puts,
        'l3_putx': l3_putx,
        'l3_inv': l3_inv,
        'l3_invx': l3_invx,
        'l3_fwd': l3_fwd,
        'l3_lat_getnl': l3_lat_getnl,
        'l3_lat_getnet': l3_lat_getnet,
        'l3_shared_requests': l3_shared_requests,
        
        # LFMR
        'lfmr': round(lfmr, 4),
        
        # Memory controller metrics
        'mem_read_requests': mem_rd,
        'mem_write_requests': mem_wr,
        'mem_total_requests': mem_rd + mem_wr,
        'mem_read_latency': mem_rdlat,
        'mem_write_latency': mem_wrlat,
        'mem_total_latency': mem_rdlat + mem_wrlat,
        'mem_avg_read_latency': round(avg_mem_read_lat, 2),
        'mem_avg_write_latency': round(avg_mem_write_lat, 2),
        'mem_reissued_accesses': mem_reissued,
        
        # Energy metrics (pJ)
        'l1i_energy_pJ': round(energy_l1i, 2),
        'l1d_energy_pJ': round(energy_l1d, 2),
        'l2_energy_pJ': round(energy_l2, 2),
        'l3_energy_pJ': round(energy_l3, 2),
        'memory_energy_pJ': round(energy_memory, 2),
        'cpu_core_energy_pJ': round(energy_cpu_core, 2),
        'total_cache_energy_pJ': round(total_cache_energy, 2),
        'total_system_energy_pJ': round(total_system_energy, 2),
        
        # Power metrics
        'average_power_mW': round(average_power_watts * 1000, 4),
        'average_power_W': round(average_power_watts, 6),
        'perf_power_ratio': round(perf_power_ratio * 1000, 4),
        
        # Energy breakdown percentages
        'l1i_energy_percent': round((energy_l1i / total_system_energy) * 100, 2) if total_system_energy > 0 else 0.0,
        'l1d_energy_percent': round((energy_l1d / total_system_energy) * 100, 2) if total_system_energy > 0 else 0.0,
        'l2_energy_percent': round((energy_l2 / total_system_energy) * 100, 2) if total_system_energy > 0 else 0.0,
        'l3_energy_percent': round((energy_l3 / total_system_energy) * 100, 2) if total_system_energy > 0 else 0.0,
        'memory_energy_percent': round((energy_memory / total_system_energy) * 100, 2),
        'cpu_energy_percent': round((energy_cpu_core / total_system_energy) * 100, 2),
    }

def main():
    print("🚀 Complete ZSim Metrics Extractor 🚀")
    print("=" * 60)
    
    # Find all .coffee files in current directory
    out_files = glob.glob("*.coffee")
    
    if not out_files:
        print("❌ No .coffee files found in current directory!")
        print("Make sure you're in the directory with your zsim output files.")
        return
    
    print(f"📁 Found {len(out_files)} .coffee files:")
    for f in out_files:
        print(f"   • {f}")
    
    # Ask for output CSV name
    csv_filename = input("\n💾 Enter output CSV filename (default: complete_metrics.csv): ").strip()
    if not csv_filename:
        csv_filename = "complete_metrics.csv"
    
    if not csv_filename.endswith('.csv'):
        csv_filename += '.csv'
    
    # Comprehensive CSV headers
    headers = [
        # Workload identification
        'Workload',
        
        # Basic execution metrics
        'Instructions', 'Cycles', 'IPC', 'Execution_Time_s', 'Execution_Time_us', 'Weave_Time',
        
        # Core performance metrics
        'UOps', 'Branch_UOps', 'FP_Add_Sub_UOps', 'FP_Mul_Div_UOps', 
        'Basic_Blocks', 'Approx_Instrs', 'Contention_Cycles', 'Contention_%',
        
        # Branch prediction metrics
        'Mispred_Branches', 'Mispred_Instrs', 'Mispred_Penalty', 
        'Branch_Pred_Accuracy_%', 'Avg_Mispred_Penalty',
        
        # Stall metrics
        'Load_Stalls', 'Store_Stalls', 'Total_Stalls', 
        'Load_Stall_%', 'Store_Stall_%',
        
        # Locality metrics
        'Spatial_Locality', 'Temporal_Locality',
        
        # L1I Cache metrics
        'L1I_Hits', 'L1I_Misses', 'L1I_Filtered_Hits', 'L1I_Total_Accesses',
        'L1I_Miss_Rate_%', 'L1I_Hit_Rate_%', 'L1I_Shared_Requests',
        
        # L1D Cache metrics
        'L1D_GETS_Hits', 'L1D_GETX_Hits', 'L1D_Total_Hits',
        'L1D_GETS_Misses', 'L1D_GETX_Misses', 'L1D_Total_Misses',
        'L1D_Filtered_GETS', 'L1D_Filtered_GETX', 'L1D_Total_Accesses',
        'L1D_Miss_Rate_%', 'L1D_Hit_Rate_%',
        'L1D_PUTS', 'L1D_PUTX', 'L1D_INV', 'L1D_INVX', 'L1D_FWD',
        'L1D_Lat_GetNL', 'L1D_Lat_GetNet', 'L1D_Shared_Requests',
        
        # Combined L1 metrics
        'L1_Total_Hits', 'L1_Total_Misses', 'L1_Total_Accesses',
        'L1_Miss_Rate_%', 'L1_Hit_Rate_%', 'L1_MPKI',
        
        # L2 Cache metrics
        'L2_GETS_Hits', 'L2_GETX_Hits', 'L2_Total_Hits',
        'L2_GETS_Misses', 'L2_GETX_IM_Misses', 'L2_GETX_SM_Misses', 'L2_Total_Misses',
        'L2_Total_Accesses', 'L2_Miss_Rate_%', 'L2_Hit_Rate_%', 'L2_MPKI',
        'L2_PUTS', 'L2_PUTX', 'L2_INV', 'L2_INVX', 'L2_FWD',
        'L2_Lat_GetNL', 'L2_Lat_GetNet', 'L2_Shared_Requests',
        
        # L3 Cache metrics
        'L3_GETS_Hits', 'L3_GETX_Hits', 'L3_Total_Hits',
        'L3_GETS_Misses', 'L3_GETX_IM_Misses', 'L3_GETX_SM_Misses', 'L3_Total_Misses',
        'L3_Total_Accesses', 'L3_Miss_Rate_%', 'L3_Hit_Rate_%', 'L3_MPKI',
        'L3_PUTS', 'L3_PUTX', 'L3_INV', 'L3_INVX', 'L3_FWD',
        'L3_Lat_GetNL', 'L3_Lat_GetNet', 'L3_Shared_Requests',
        
        # LFMR
        'LFMR',
        
        # Memory controller metrics
        'Mem_Read_Requests', 'Mem_Write_Requests', 'Mem_Total_Requests',
        'Mem_Read_Latency', 'Mem_Write_Latency', 'Mem_Total_Latency',
        'Mem_Avg_Read_Latency', 'Mem_Avg_Write_Latency', 'Mem_Reissued_Accesses',
        
        # Energy metrics (pJ)
        'L1I_Energy_pJ', 'L1D_Energy_pJ', 'L2_Energy_pJ', 'L3_Energy_pJ',
        'Memory_Energy_pJ', 'CPU_Core_Energy_pJ', 
        'Total_Cache_Energy_pJ', 'Total_System_Energy_pJ',
        
        # Power metrics
        'Average_Power_mW', 'Average_Power_W', 'Perf_Power_Ratio_IPC_per_mW',
        
        # Energy breakdown percentages
        'L1I_Energy_%', 'L1D_Energy_%', 'L2_Energy_%', 'L3_Energy_%',
        'Memory_Energy_%', 'CPU_Energy_%'
    ]
    
    # Process all files
    print(f"\n{'='*60}")
    print("Processing files...")
    print(f"{'='*60}\n")
    
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        
        for i, out_file in enumerate(out_files, 1):
            print(f"⚡ Processing {i}/{len(out_files)}: {out_file}")
            
            try:
                # Generate workload name from filename
                workload_name = Path(out_file).stem
                
                # Parse the file
                metrics = parse_zsim_file(out_file)
                
                # Prepare data row
                row = [
                    workload_name,
                    
                    # Basic execution metrics
                    metrics['instructions'], metrics['cycles'], metrics['ipc'], 
                    metrics['execution_time_s'], metrics['execution_time_us'], metrics['weave_time'],
                    
                    # Core performance metrics
                    metrics['uops'], metrics['branch_uops'], metrics['fp_add_sub_uops'], 
                    metrics['fp_mul_div_uops'], metrics['bbls'], metrics['approx_instrs'],
                    metrics['contention_cycles'], metrics['contention_percentage'],
                    
                    # Branch prediction metrics
                    metrics['mispred_branches'], metrics['mispred_instrs'], metrics['mispred_penalty'],
                    metrics['branch_pred_accuracy'], metrics['avg_mispred_penalty'],
                    
                    # Stall metrics
                    metrics['load_stalls'], metrics['store_stalls'], metrics['total_stalls'],
                    metrics['load_stall_percentage'], metrics['store_stall_percentage'],
                    
                    # Locality metrics
                    metrics['spatial_locality'], metrics['temporal_locality'],
                    
                    # L1I Cache metrics
                    metrics['l1i_hits'], metrics['l1i_misses'], metrics['l1i_filtered_hits'],
                    metrics['l1i_total_accesses'], metrics['l1i_miss_rate'], metrics['l1i_hit_rate'],
                    metrics['l1i_shared_requests'],
                    
                    # L1D Cache metrics
                    metrics['l1d_gets_hits'], metrics['l1d_getx_hits'], metrics['l1d_total_hits'],
                    metrics['l1d_gets_misses'], metrics['l1d_getx_misses'], metrics['l1d_total_misses'],
                    metrics['l1d_filtered_gets'], metrics['l1d_filtered_getx'], 
                    metrics['l1d_total_accesses'], metrics['l1d_miss_rate'], metrics['l1d_hit_rate'],
                    metrics['l1d_puts'], metrics['l1d_putx'], metrics['l1d_inv'], 
                    metrics['l1d_invx'], metrics['l1d_fwd'],
                    metrics['l1d_lat_getnl'], metrics['l1d_lat_getnet'], metrics['l1d_shared_requests'],
                    
                    # Combined L1 metrics
                    metrics['l1_total_hits'], metrics['l1_total_misses'], metrics['l1_total_accesses'],
                    metrics['l1_miss_rate'], metrics['l1_hit_rate'], metrics['l1_mpki'],
                    
                    # L2 Cache metrics
                    metrics['l2_gets_hits'], metrics['l2_getx_hits'], metrics['l2_total_hits'],
                    metrics['l2_gets_misses'], metrics['l2_getx_im_misses'], 
                    metrics['l2_getx_sm_misses'], metrics['l2_total_misses'],
                    metrics['l2_total_accesses'], metrics['l2_miss_rate'], 
                    metrics['l2_hit_rate'], metrics['l2_mpki'],
                    metrics['l2_puts'], metrics['l2_putx'], metrics['l2_inv'], 
                    metrics['l2_invx'], metrics['l2_fwd'],
                    metrics['l2_lat_getnl'], metrics['l2_lat_getnet'], metrics['l2_shared_requests'],
                    
                    # L3 Cache metrics
                    metrics['l3_gets_hits'], metrics['l3_getx_hits'], metrics['l3_total_hits'],
                    metrics['l3_gets_misses'], metrics['l3_getx_im_misses'], 
                    metrics['l3_getx_sm_misses'], metrics['l3_total_misses'],
                    metrics['l3_total_accesses'], metrics['l3_miss_rate'], 
                    metrics['l3_hit_rate'], metrics['l3_mpki'],
                    metrics['l3_puts'], metrics['l3_putx'], metrics['l3_inv'], 
                    metrics['l3_invx'], metrics['l3_fwd'],
                    metrics['l3_lat_getnl'], metrics['l3_lat_getnet'], metrics['l3_shared_requests'],
                    
                    # LFMR
                    metrics['lfmr'],
                    
                    # Memory controller metrics
                    metrics['mem_read_requests'], metrics['mem_write_requests'], 
                    metrics['mem_total_requests'],
                    metrics['mem_read_latency'], metrics['mem_write_latency'], 
                    metrics['mem_total_latency'],
                    metrics['mem_avg_read_latency'], metrics['mem_avg_write_latency'],
                    metrics['mem_reissued_accesses'],
                    
                    # Energy metrics
                    metrics['l1i_energy_pJ'], metrics['l1d_energy_pJ'], 
                    metrics['l2_energy_pJ'], metrics['l3_energy_pJ'],
                    metrics['memory_energy_pJ'], metrics['cpu_core_energy_pJ'],
                    metrics['total_cache_energy_pJ'], metrics['total_system_energy_pJ'],
                    
                    # Power metrics
                    metrics['average_power_mW'], metrics['average_power_W'], 
                    metrics['perf_power_ratio'],
                    
                    # Energy breakdown percentages
                    metrics['l1i_energy_percent'], metrics['l1d_energy_percent'],
                    metrics['l2_energy_percent'], metrics['l3_energy_percent'],
                    metrics['memory_energy_percent'], metrics['cpu_energy_percent']
                ]
                
                writer.writerow(row)
                
                # Show quick stats
                print(f"   ✅ IPC: {metrics['ipc']}")
                print(f"      Power: {metrics['average_power_mW']} mW")
                print(f"      Perf/Power: {metrics['perf_power_ratio']} IPC/mW")
                print(f"      L1 Miss Rate: {metrics['l1_miss_rate']}%")
                print(f"      L2 Miss Rate: {metrics['l2_miss_rate']}%")
                print(f"      L3 Miss Rate: {metrics['l3_miss_rate']}%")
                print(f"      Branch Pred Accuracy: {metrics['branch_pred_accuracy']}%")
                print()
                
            except Exception as e:
                print(f"   ❌ Error processing {out_file}: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    print(f"{'='*60}")
    print(f"🎉 Done! All results saved to: {csv_filename}")
    print(f"📊 Processed {len(out_files)} files successfully!")
    print(f"{'='*60}")
    print(f"\n📈 Metrics extracted:")
    print(f"   • Basic Execution: Instructions, Cycles, IPC, Execution Time")
    print(f"   • Core Performance: UOps, Branch Ops, FP Ops, Basic Blocks")
    print(f"   • Branch Prediction: Accuracy, Mispredictions, Penalties")
    print(f"   • Stalls: Load/Store stalls with percentages")
    print(f"   • Cache Hierarchy: Complete L1I/L1D/L2/L3 statistics")
    print(f"   • Memory: Read/Write requests and latencies")
    print(f"   • Energy: Per-component energy breakdown with percentages")
    print(f"   • Power: Average power and performance/power ratio")
    print(f"\n💡 You can now open {csv_filename} in Excel/LibreOffice or analyze with Python/R")
    print(f"💡 Total of {len(headers)} columns extracted per workload!")

if __name__ == "__main__":
    main()

🚀 Complete ZSim Metrics Extractor 🚀
📁 Found 5 .coffee files:
   • adaboost_data_loading_zsim_pim.coffee
   • adaboost_host_moore_zsim.coffee
   • adaboost_pim_compute_zsim.coffee
   • adaboost_pim_full_real_ryzen_zsim.coffee
   • adaboost_zsim_pim_network.coffee

Processing files...

⚡ Processing 1/5: adaboost_data_loading_zsim_pim.coffee
   ✅ IPC: 1.3833
      Power: 3.8353 mW
      Perf/Power: 360675.0866 IPC/mW
      L1 Miss Rate: 2.22%
      L2 Miss Rate: 0.0%
      L3 Miss Rate: 0.0%
      Branch Pred Accuracy: 99.41%

⚡ Processing 2/5: adaboost_host_moore_zsim.coffee
   ✅ IPC: 1.4168
      Power: 7.8155 mW
      Perf/Power: 181287.0041 IPC/mW
      L1 Miss Rate: 1.1%
      L2 Miss Rate: 17.29%
      L3 Miss Rate: 94.01%
      Branch Pred Accuracy: 99.41%

⚡ Processing 3/5: adaboost_pim_compute_zsim.coffee
   ✅ IPC: 1.3884
      Power: 3.7795 mW
      Perf/Power: 367347.3261 IPC/mW
      L1 Miss Rate: 1.98%
      L2 Miss Rate: 0.0%
      L3 Miss Rate: 0.0%
      Branch Pred Accura